In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

In [20]:
n, m = 5, 3
k, l =2, 1

# A = np.random.random((n, (l+1-k)*n))
A = np.random.random((n, m))
n, m = A.shape
n, m, k, l

(5, 3, 2, 1)

In [22]:
# A_ub_1
# A_ub_1 = np.zeros((n, n*m))
rows = np.repeat(np.arange(n), m)
columns = np.arange(n*m)
# A_ub_1[rows, columns] = 1
# A_ub_2
A_ub_2 = np.zeros((n, n*m))
A_ub_2[rows, columns] = -1
# A_ub_3
# A_ub_3 = -1. * np.tile(np.eye(m), n)
A_ub_3 = 1. * np.tile(np.eye(m), n)
# A_ub
# A_ub = np.vstack([A_ub_1, A_ub_2, A_ub_3])
A_ub = np.vstack([A_ub_2, A_ub_3])
A_ub

array([[-1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
        -1., -1.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,
         0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  1.]])

In [23]:
n,m

(5, 3)

In [24]:
# b_ub = np.hstack([np.repeat(k, n), np.repeat(-1, n), np.repeat(-l, m)])
b_ub = np.hstack([np.repeat(-l, n), np.repeat(k, m)])

In [25]:
try:
    model = gp.Model('gp')

    x = model.addMVar(shape = n*m, vtype = GRB.BINARY, name = 'x')
    # x = model.addMVar(shape = n*m, vtype = GRB.INTEGER, name = 'x')
    # x = model.addMVar(shape = n*m, name = 'x')

    obj = A.flatten()
    model.setObjective(obj @ x, GRB.MINIMIZE)
    A_ub_sp = sp.csr_matrix(A_ub)
    rhs = b_ub
    model.addConstr(A_ub_sp @ x <= rhs, name = "c")
    # model.addConstr(x>=0, name = "positive")

    model.optimize()
    # print(x.X)
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 2700 Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 15 columns and 30 nonzeros
Model fingerprint: 0xb2bc81a9
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 2.5251883
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 30 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
Found heuristic solution: objective 2.0090168

Root relaxation: objective 1.131735e+00, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     

In [26]:
X = x.X.reshape([n,m])

In [28]:
k, l

(2, 1)

In [30]:
A

array([[0.42073612, 0.46886166, 0.14353377],
       [0.25955207, 0.15723849, 0.58574513],
       [0.52387789, 0.18823528, 0.76303762],
       [0.13723227, 0.85356951, 0.73019178],
       [0.50549532, 0.45417486, 0.66637247]])

In [29]:
X

array([[-0., -0.,  1.],
       [-0.,  1., -0.],
       [-0.,  1., -0.],
       [ 1., -0., -0.],
       [ 1.,  0., -0.]])

In [121]:
X.sum(1), X.sum(0)

(array([1., 1., 1., 1., 1., 1., 1.]), array([2., 5.]))